Import necessary packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

Import datasets and packages

In [2]:
df_lookup = pd.read_csv('../../data/raw/EXTR_LookUp.csv', dtype='str')
#df_parcel = pd.read_csv('../../data/raw/EXTR_PARCEL.csv', dtype='str')
df_resbldg = pd.read_csv('../../data/raw/EXTR_ResBldg.csv', dtype='str')
df_rpsale = pd.read_csv('../../data/raw/EXTR_RPSale.csv', dtype='str')

Strip leading and trailing spaces

In [3]:
from custom_functions import strip_spaces

df_lookup = strip_spaces(df_lookup)
#df_parcel = strip_spaces(df_parcel)
df_resbldg = strip_spaces(df_resbldg)
df_rpsale = strip_spaces(df_rpsale)

Drop columns that are obviously unnecessary

In [4]:
resbldg_desired_columns = ['Major', 'Minor', 'NbrLivingUnits', 'ZipCode', 'Stories', 'BldgGrade', 
                           'BldgGradeVar', 'SqFt1stFloor', 'SqFtHalfFloor', 'SqFt2ndFloor',
                           'SqFtUpperFloor', 'SqFtUnfinFull', 'SqFtUnfinHalf', 'SqFtTotLiving', 'SqFtTotBasement', 
                           'SqFtFinBasement', 'FinBasementGrade', 'SqFtGarageBasement', 'SqFtGarageAttached', 
                           'DaylightBasement','SqFtOpenPorch', 'SqFtEnclosedPorch', 'SqFtDeck', 'HeatSystem',
                           'HeatSource', 'BrickStone', 'ViewUtilization', 'Bedrooms','BathHalfCount', 
                           'Bath3qtrCount', 'BathFullCount', 'FpSingleStory','FpMultiStory', 'FpFreestanding', 
                           'FpAdditional', 'YrBuilt','YrRenovated', 'PcntComplete', 'Obsolescence', 
                           'PcntNetCondition','Condition']
df_resbldg = df_resbldg[resbldg_desired_columns].copy()

rpsale_desired_columns = ['ExciseTaxNbr', 'Major', 'Minor', 'DocumentDate', 'SalePrice', 'RecordingNbr', 'PropertyType', 
                          'PrincipalUse', 'SaleInstrument', 'AFForestLand', 'AFCurrentUseLand', 'AFNonProfitUse', 
                          'AFHistoricProperty', 'SaleReason', 'PropertyClass', 'SaleWarning']
df_rpsale = df_rpsale[rpsale_desired_columns].copy()

## Refine *Sales* DataFrame

**Eliminate irrelevant property types, non-2019 sales, and add necessary helper columns**

In [5]:
res_sales = df_rpsale.copy()

principal_use_codes_to_keep = ['6'] # Consider investigating '4' too 
principal_use_codes_to_drop = np.setdiff1d(res_sales.PrincipalUse.unique(), principal_use_codes_to_keep)

property_class_codes_to_keep = ['8']
property_class_codes_to_drop = np.setdiff1d(res_sales.PropertyClass.unique(), property_class_codes_to_keep)

# Need to consider where to classify codes like 2,3 since there isn't an explicit statement
#    - Possible to disregard entirely, explore data first
# Consider looking into single-family and multiple-family separately
property_type_codes_to_keep =['11']# ['2', '3', '6', '10', '11', '12', '13', '18', '19']
property_type_codes_to_drop = np.setdiff1d(res_sales.PropertyType.unique(), property_type_codes_to_keep)

for code in principal_use_codes_to_drop:
    res_sales['PrincipalUse'].replace(to_replace=code, value=np.nan, inplace=True)

for code in property_class_codes_to_drop:
    res_sales['PropertyClass'].replace(to_replace=code, value=np.nan, inplace=True)

for code in property_type_codes_to_drop:
    res_sales['PropertyType'].replace(to_replace=code, value=np.nan, inplace=True)

# CREATE PARCEL ID
res_sales['Parcel_ID'] = res_sales.Major + '-' + res_sales.Minor


# KEEP ONLY 2019 SALES
res_sales['DocumentDate'] = res_sales.DocumentDate.astype(np.datetime64)
res_sales['SaleYear'] = [sale.year for sale in res_sales['DocumentDate']]
res_sales = res_sales.loc[res_sales['SaleYear']==2019].copy()


# ELIMINATE SALES OF ZERO DOLLARS
res_sales['SalePrice'] = res_sales.SalePrice.astype('int')
res_sales['SalePrice'].replace(0, np.nan, inplace=True)


# DROP SALES DETERMINED TO BE INVALID
res_sales.dropna(inplace=True)


# CREATE COLUMN TO IDENTIFY DUPLICATES
res_sales['SaleCount'] = list(map(dict(res_sales.Parcel_ID.value_counts()).get, res_sales.Parcel_ID))

**Remove sales for a given property that are not the most recent**

This prevents re-sale of homes from counting for multiple entries, which would overrepresent homes that tend to be resold within the final model. More importantly, it is a necessary step in order to join the Sales database with the Residential Building database

In [6]:
def identify_latest_sale(docdates, parcel_ids):
    latest_parcel_sale = []
    data = pd.DataFrame([docdates, parcel_ids]).T
    data.DocumentDate = data.DocumentDate.astype('datetime64')
 
    for i, parcel_id in enumerate(data.Parcel_ID):
        relevant_docdates = data.loc[data.Parcel_ID == parcel_id, 'DocumentDate']
        max_docdate = relevant_docdates.values.max()
        
        this_datetime = np.datetime64(data.iloc[i, 0]) 
        latest_parcel_sale.append(this_datetime == max_docdate)

    return latest_parcel_sale

tf = identify_latest_sale(res_sales.DocumentDate, res_sales.Parcel_ID)
latest_sales = res_sales.loc[tf].copy()
latest_sales['SaleCount'] = list(map(dict(latest_sales.Parcel_ID.value_counts()).get, latest_sales.Parcel_ID))

**Determine average price for multiple sales on the same day on the same parcel**

This is a necessary step to joining the Sales database with the Residential Building database by removing duplicate entries without losing the valuable sales price data that would be lost by dropping a duplicate at random.

In [7]:
def avg_price_for_duped_parcels(data):
    dupes = data.loc[data.SaleCount > 1]
    for i, ind in enumerate(dupes.index):
        parcel_id = data.loc[ind, 'Parcel_ID']
        parcels_w_parcel_id = data.loc[data.Parcel_ID == parcel_id, 'SalePrice']

        avg_price_for_id = parcels_w_parcel_id.values.mean()
        for parcel_index in parcels_w_parcel_id.index:
            data.at[parcel_index, 'SalePrice'] = avg_price_for_id
    return data

# Average pricing for duplicates
latest_sales_averaged = avg_price_for_duped_parcels(latest_sales)
latest_sales_averaged['SaleCount'] = list(map(dict(latest_sales_averaged.Parcel_ID.value_counts()).get, latest_sales_averaged.Parcel_ID))


# Remove duplicates
latest_sales_averaged.index = latest_sales_averaged.Parcel_ID.values
latest_sales_averaged_deduped = latest_sales_averaged.drop_duplicates('Parcel_ID')
latest_sales_averaged_deduped.reset_index(inplace=True, drop=True)
latest_sales_averaged_deduped.index = latest_sales_averaged_deduped.Parcel_ID.values


# Drop unused columns as final step of cleaning before join
latest_sales_averaged_deduped_tokeep = ['SalePrice', 'Parcel_ID', 'PropertyType', 'PrincipalUse', 
                                        'SaleInstrument', 'AFForestLand', 'AFCurrentUseLand', 
                                        'AFNonProfitUse', 'AFHistoricProperty', 'SaleReason', 
                                        'PropertyClass', 'SaleWarning']
sales = latest_sales_averaged_deduped[latest_sales_averaged_deduped_tokeep].copy()


sales.head()

,SalePrice,Parcel_ID,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning
213043-0120,560000.0,213043-0120,11,6,3,N,N,N,N,1,8,
940652-0630,435000.0,940652-0630,11,6,3,N,N,N,N,1,8,
347050-0040,648500.0,347050-0040,11,6,3,N,N,N,N,1,8,
345960-0230,1255000.0,345960-0230,11,6,3,N,N,N,N,1,8,40
030200-0645,380000.0,030200-0645,11,6,3,N,N,N,N,1,8,10


# Handle Residential Building dataframe

**Create DataFrame for Porch metrics**

In [8]:
df_resbldg.head(2)

,Major,Minor,NbrLivingUnits,ZipCode,Stories,BldgGrade,BldgGradeVar,SqFt1stFloor,SqFtHalfFloor,SqFt2ndFloor,SqFtUpperFloor,SqFtUnfinFull,SqFtUnfinHalf,SqFtTotLiving,SqFtTotBasement,SqFtFinBasement,FinBasementGrade,SqFtGarageBasement,SqFtGarageAttached,DaylightBasement,SqFtOpenPorch,SqFtEnclosedPorch,SqFtDeck,HeatSystem,HeatSource,BrickStone,ViewUtilization,Bedrooms,BathHalfCount,Bath3qtrCount,BathFullCount,FpSingleStory,FpMultiStory,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition
0,009800,0720,1,98075,2,11,0,1970,0,2130,0,0,0,4100,0,0,0,0,750,,0,0,0,5,2,0,,4,1,0,3,3,0,0,0,2001,0,0,0,0,3
1,009802,0140,1,98075,2,10,0,1610,0,1400,0,0,0,3010,0,0,0,0,660,,380,0,0,5,2,0,,4,1,2,2,3,0,0,0,2004,0,0,0,0,3


Basic preparation of residential building database

In [9]:
df_resbldg['Parcel_ID'] = df_resbldg['Major'] + '-' + df_resbldg['Minor']
df_resbldg['SqFtOpenPorch'] = df_resbldg['SqFtOpenPorch'].astype('int')
df_resbldg['SqFtEnclosedPorch'] = df_resbldg['SqFtEnclosedPorch'].astype('int')
df_resbldg['Bedrooms'] = df_resbldg['Bedrooms'].astype('int')
df_resbldg['SqFtGarageAttached'] = df_resbldg['SqFtGarageAttached'].astype('int')
df_resbldg['SqFtGarageBasement'] = df_resbldg['SqFtGarageBasement'].astype('int')
bedrooms = df_resbldg['Bedrooms'].astype('int')

One-hot encode porch metrics

In [10]:
# df_resbldg['porch_both'] = (df_resbldg.SqFtOpenPorch > 0) & (df_resbldg.SqFtEnclosedPorch > 0)
# df_resbldg['porch_none'] = (df_resbldg.SqFtOpenPorch == 0) & (df_resbldg.SqFtEnclosedPorch == 0)
# df_resbldg['porch_closed'] = df_resbldg.SqFtEnclosedPorch > 0
# df_resbldg['porch_open'] = df_resbldg.SqFtOpenPorch > 0

One-hot encode heating metrics

In [11]:
# from custom_functions import one_hot
# from custom_functions import get_lookups

# heating = one_hot(df_resbldg['HeatSystem'], name_lookup = get_lookups(108, df_lookup), prefix='heat')
# df_resbldg = pd.concat([df_resbldg, heating], axis=1)

### Join with SQL and Export

In [12]:
df_resbldg.columns

Index(['Major', 'Minor', 'NbrLivingUnits', 'ZipCode', 'Stories', 'BldgGrade',
       'BldgGradeVar', 'SqFt1stFloor', 'SqFtHalfFloor', 'SqFt2ndFloor',
       'SqFtUpperFloor', 'SqFtUnfinFull', 'SqFtUnfinHalf', 'SqFtTotLiving',
       'SqFtTotBasement', 'SqFtFinBasement', 'FinBasementGrade',
       'SqFtGarageBasement', 'SqFtGarageAttached', 'DaylightBasement',
       'SqFtOpenPorch', 'SqFtEnclosedPorch', 'SqFtDeck', 'HeatSystem',
       'HeatSource', 'BrickStone', 'ViewUtilization', 'Bedrooms',
       'BathHalfCount', 'Bath3qtrCount', 'BathFullCount', 'FpSingleStory',
       'FpMultiStory', 'FpFreestanding', 'FpAdditional', 'YrBuilt',
       'YrRenovated', 'PcntComplete', 'Obsolescence', 'PcntNetCondition',
       'Condition', 'Parcel_ID'],
      dtype='object')

In [13]:
import sqlite3

conn = sqlite3.connect('../../data/processed/main.db', )
df_resbldg.to_sql('buildings', conn, if_exists='replace')
sales.to_sql('sales', conn, if_exists='replace')

q = ''' SELECT * FROM buildings
LEFT JOIN sales USING (Parcel_ID)'''

joined = pd.read_sql(q, conn)

keepers = ['SalePrice', 'Parcel_ID','NbrLivingUnits', 'ZipCode', 'Stories', 'BldgGrade', 'BldgGradeVar', 'SqFt1stFloor', 'SqFtHalfFloor', 'SqFt2ndFloor', 
'SqFtUpperFloor', 'SqFtUnfinFull', 'SqFtUnfinHalf', 'SqFtTotLiving', 'SqFtTotBasement', 'SqFtFinBasement', 'FinBasementGrade', 'SqFtGarageBasement', 
'SqFtGarageAttached', 'DaylightBasement', 'SqFtOpenPorch', 'SqFtEnclosedPorch', 'SqFtDeck', 'HeatSystem', 'HeatSource', 'BrickStone', 'ViewUtilization', 
'Bedrooms', 'BathHalfCount', 'Bath3qtrCount', 'BathFullCount', 'FpSingleStory', 'FpMultiStory', 'FpFreestanding', 'FpAdditional', 'YrBuilt', 
 'YrRenovated', 'PcntComplete', 'Obsolescence', 'PcntNetCondition', 'Condition', 'PropertyType', 'PrincipalUse', 'SaleInstrument', 'AFForestLand', 
 'AFCurrentUseLand', 'AFNonProfitUse', 'AFHistoricProperty', 'SaleReason', 'PropertyClass', 'SaleWarning']
df_main = joined[keepers].copy()

df_main.dropna(inplace=True)
df_main.reset_index(inplace=True, drop=True)

df_main.to_sql('df_main', conn, if_exists='replace')
conn.close()